In [2]:
import csv
import requests
import pandas as pd
import json
import logging

# Set up basic configuration for logging
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_assignment_data_for_student_concurrent(student_id, base_url, token):
    headers = {"Authorization": f"Bearer {token}"}
    full_url = base_url.format(student_id)
    try:
        response = requests.get(full_url, headers=headers)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        assignment_data = response.json()  # Assign the JSON response data to assignment_data
        # Debug output
        #print(f"Data retrieved for student {student_id}: {assignment_data}")
        return student_id, assignment_data        
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch data for student {student_id}: {e}")
        return student_id, None

def fetch_data_and_save_to_excel_concurrent(csv_path, base_url, token, json_file, excel_file, max_workers=10):
    data = []
    student_ids = []

    with open(csv_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        student_id_list = [row['Student ID'] for row in reader]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_student_id = {executor.submit(fetch_assignment_data_for_student_concurrent, student_id, base_url, token): student_id for student_id in student_id_list}
        
        for future in as_completed(future_to_student_id):
            student_id, assignment_data = future.result()
            if assignment_data is not None:
                data.extend(assignment_data)
                student_ids.extend([student_id] * len(assignment_data))
            else:
                print(f"No data fetched for student ID: {student_id}.")

    with open(json_file, 'w') as jsonfile:
        json.dump(data, jsonfile)
    
    if data:  # Proceed only if data is not empty
        df = pd.DataFrame(data)
        df['Student ID'] = student_ids
        cols = df.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        df = df[cols]
        df.to_excel(excel_file, index=False)
    else:
        print("No data to save.")

# Here's where you replace the placeholders with your actual values:
csv_file_path = 'student_roster.csv'  # Replace 'path/to/your/student_roster.csv' with the actual path to your CSV file.
api_base_url = "https://usflearn.instructure.com/api/v1/courses/1857108/analytics/users/{}/assignments"  # Use your actual API base URL.
bearer_token = "13~D9Rs8zX2eEumEXC0knPDXdIXQLbq4cKVwVnYGmBYEgBFPMI2Yu1suCAk1l1E90Uq"  # Use your actual bearer token.
output_json_file = 'assignment_data.json'  # Specify your desired filename or path for the JSON output.
output_excel_file = 'assignment_data.xlsx'  # Specify your desired filename or path for the Excel output.

# Finally, call the function with the parameters you've set.
fetch_data_and_save_to_excel_concurrent(
    csv_path=csv_file_path,
    base_url=api_base_url,
    token=bearer_token,
    json_file=output_json_file,
    excel_file=output_excel_file
)


In [3]:
import pandas as pd

# Load the Excel file into a pandas DataFrame
df = pd.read_excel('assignment_data.xlsx')  # Update this path to your actual file path

# Get unique student IDs
unique_student_ids = df['Student ID'].unique()

# Count the number of unique student IDs
unique_student_id_count = len(unique_student_ids)

print(f"Unique Student IDs: {unique_student_ids}")
print(f"Count of Unique Student IDs: {unique_student_id_count}")


Unique Student IDs: [5037219 5071983 5036067 5120753 5127901 5096027 5114089 5074101 4900087
 5287385 5023795 4812901 5050949 5046985 4791905 5112297 5079999 5054947
 5061527 5053211 5049551 5022873 4821299 4743301 5042671 5043529 5044659
 4930051 5054799 4929257 5125835 5051901 5070843 5026003 5032527 4953475
 5094475 4696765 5043651 4884777 4992671 4940821 5072481 5045021 5006045
 5098345 5016213 5050005 5001727 5139959 4790815 4931121 4830019 5023151
 4712135 5052269 5016829 5032939 5063607 4827613 5012351 4633299 5053455
 5040615 5040809 5144913 5094291 4689747 5072573 5050875 4897609 4817509
 5042845 4786405 4919819 5042545 4841879 4857797 4900575 4836427 5022395
 5025957 5079511 5127993 5094025 5128729 4853409 5060935 4977023 5064579
 4534855 5008163 4855855 5112863 5056459 5090351 5011369 4827379 5082597
 5064081 5036345 5085943 4837299 5114903 4951429 5054323 5182735 4925287
 5092269 5009577 5054235 4311007 5087555 5073473 5078327 4159521 4954877
 4830499 5075811 4808169 467898